<a href="https://colab.research.google.com/github/WESTZERO115/childmeal/blob/main/lack_intake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymysql

     |████████████████████████████████| 43 kB 810 kB/s 


1. user의 식사 내역 불러오기\
==> user의 카드 번호 정보 cardnum 변수로 저장\
==> user가 먹은 메뉴 eat_menu 변수로 저장

In [7]:
import pymysql

conn=pymysql.connect(host='capstonedesign.c3lmane8vo1i.ap-northeast-2.rds.amazonaws.com',
                        port=3306,
                        user='admin',
                        password='swusw2022',
                        db='capstonedesign',
                        charset='utf8')
cursor=conn.cursor()
cursor.execute("SELECT * FROM EATMEAL;") 
(ChildInfo, food_menu, date_time) = cursor.fetchone()
cardnum = ChildInfo
eat_menu = food_menu 
print(f"{ChildInfo},{food_menu},{date_time}")
conn.commit()
conn.close()

678765456545,김밥,2022-04-28 06:25:10


2. cardnum 정보를 가지고 CHILD DB에서 childage 정보 불러오기

In [8]:
conn=pymysql.connect(host='capstonedesign.c3lmane8vo1i.ap-northeast-2.rds.amazonaws.com',
                        port=3306,
                        user='admin',
                        password='swusw2022',
                        db='capstonedesign',
                        charset='utf8')
cursor=conn.cursor()
cursor.execute("SELECT * FROM CHILD WHERE CardNum = %s;", (cardnum)) 
(CardNum, ChildName, ChildAge, ChildHeight, ChildWeight, ChildGender) = cursor.fetchone()
print(f"{CardNum},{ChildName},{ChildAge},{ChildHeight},{ChildWeight},{ChildGender}")
childage = ChildAge
conn.commit()
conn.close()

678765456545,무야호,11,136,40,남성


3. 식품 영양 정보(FOOD_NUTRITION) DB에서 user가 먹은 eat_menu의 영양 정보를 가져온다.

In [9]:
conn=pymysql.connect(host='capstonedesign.c3lmane8vo1i.ap-northeast-2.rds.amazonaws.com',
                        port=3306,
                        user='admin',
                        password='swusw2022',
                        db='capstonedesign',
                        charset='utf8')
cursor=conn.cursor()
cursor.execute("SELECT * FROM FOOD_NUTRITION WHERE food_menu = '" + eat_menu + "';")
(food_code,food_menu,food_groupname,food_servingsize,food_kcal,food_carbohyd,food_protein,food_fat,food_sugars,food_natrium,food_cholesterol,food_saturatedfatty,food_transfat) = cursor.fetchone() 
eat_kcal = food_kcal
eat_carbonhyd = food_carbohyd
eat_protein = food_protein
eat_fat = food_fat
eat_natrium = food_natrium
print(f"{food_code},{food_menu},{food_groupname},{food_servingsize},{food_kcal},{food_carbohyd},{food_protein},{food_fat},{food_sugars},{food_natrium},{food_cholesterol},{food_saturatedfatty},{food_transfat}")
conn.commit()
conn.close()

D018190,김밥,식품군 정보없음,230,322.56,45.95,11.14,10.47,0.0,705.16,44.4,2.54,0.05


4. 받아 온 user의 나이 정보를 바탕으로 연령별 하루 권장 섭취량(RECOMMEND_INTAKE)에서\
 칼로리/탄수화물/단백질/지방/나트륨에 관한 정보 가져와서 부족 영양소 계산하기

In [12]:
conn=pymysql.connect(host='capstonedesign.c3lmane8vo1i.ap-northeast-2.rds.amazonaws.com',
                        port=3306,
                        user='admin',
                        password='swusw2022',
                        db='capstonedesign',
                        charset='utf8')
cursor=conn.cursor()
cursor.execute("SELECT * FROM RECOMMENDED_INTAKE WHERE Child_Age = %s;",(childage))
(Child_Age, food_kcal, food_protein, food_fat, food_carbohyd, food_natrium) = cursor.fetchone() 
print(f"{Child_Age},{food_kcal},{food_carbohyd},{food_protein},{food_fat},{food_natrium}")
lack_kcal = food_kcal - eat_kcal
lack_carbonhyd = food_carbohyd - eat_carbonhyd
lack_protein = food_protein - eat_protein
lack_fat = food_fat - eat_fat
lack_natrium = food_natrium - eat_natrium
conn.commit()
conn.close()

11,1802.1,48.26,62.95,62.95,2422.99


5. 부족한 영양소 출력

In [6]:
print("오늘 하루 부족한 칼로리는 %.2f kcal 입니다."%(lack_kcal))
print("오늘 하루 부족한 탄수화물은 %.2f g 입니다."%(lack_carbonhyd))
print("오늘 하루 부족한 단백질은 %.2f g 입니다."%(lack_protein))
print("오늘 하루 부족한 지방은 %.2f g 입니다."%(lack_fat))
print("오늘 하루 부족한 나트륨은 %.2f mg 입니다."%(lack_natrium))

오늘 하루 부족한 칼로리는 1479.54 kcal 입니다.
오늘 하루 부족한 탄수화물은 2.31 g 입니다.
오늘 하루 부족한 단백질은 51.81 g 입니다.
오늘 하루 부족한 지방은 52.48 g 입니다.
오늘 하루 부족한 나트륨은 1717.83 mg 입니다.
